### Creating and Persisting an ML Model

In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
import matplotlib
from __future__ import print_function

In [2]:
df = pd.read_csv('data/ProductionData.csv', sep=',')

summary of the data 

In [3]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.00000
mean,16.692650,2.73175,2.544250,1.460450,2.032650,0.369350,3.92070,3.209450,3.062550,1.485700,2.280150,3.515800,7.40665,10.325300,10.120300,9.82185
std,1.277015,1.09966,1.090184,0.726266,0.833438,0.786741,0.90319,1.026564,1.123966,0.892544,1.280097,1.391275,9.94318,3.766443,4.188055,4.95591
min,15.000000,0.00000,0.000000,1.000000,1.000000,0.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000,1.000000,-2.000000,-2.00000
25%,16.000000,2.00000,2.000000,1.000000,1.000000,0.000000,4.00000,3.000000,2.000000,1.000000,1.000000,3.000000,2.00000,8.000000,7.000000,7.00000
50%,17.000000,3.00000,3.000000,1.000000,2.000000,0.000000,4.00000,3.000000,3.000000,1.000000,2.000000,4.000000,4.00000,10.000000,10.000000,10.00000
75%,18.000000,4.00000,3.000000,2.000000,2.000000,0.000000,5.00000,4.000000,4.000000,2.000000,3.000000,5.000000,9.00000,13.000000,13.000000,13.00000
max,22.000000,4.00000,4.000000,4.000000,4.000000,3.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,75.00000,20.000000,20.000000,21.00000


In [4]:
df.info

<bound method DataFrame.info of       school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0         GP   M   16       U     LE3       T     3     2  services     other   
1         GP   F   15       R     GT3       T     2     2     other    health   
2         GP   M   18       U     GT3       T     4     2   teacher     other   
3         GP   M   16       U     GT3       T     1     1   at_home     other   
4         GP   M   19       U     GT3       A     3     3  services  services   
...      ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
19995     GP   M   17       U     GT3       A     3     2  services     other   
19996     GP   F   16       U     LE3       T     3     3     other   teacher   
19997     GP   M   18       U     GT3       T     2     2  services    health   
19998     GP   F   18       U     GT3       T     2     2   at_home     other   
19999     GP   F   16       U     GT3       T     1     3     other     other

In [5]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,M,16,U,LE3,T,3,2,services,other,...,3,3,2,4,5,5,7,13,14,14
1,GP,F,15,R,GT3,T,2,2,other,health,...,5,4,5,1,1,4,7,8,8,9
2,GP,M,18,U,GT3,T,4,2,teacher,other,...,4,2,2,1,2,1,2,16,15,15
3,GP,M,16,U,GT3,T,1,1,at_home,other,...,5,2,3,1,2,2,1,13,13,13
4,GP,M,19,U,GT3,A,3,3,services,services,...,3,3,1,1,4,3,2,12,12,12


The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [6]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

qual_student gets added

In [7]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,qual_student
count,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.00000,20000.000000
mean,16.692650,2.73175,2.544250,1.460450,2.032650,0.369350,3.92070,3.209450,3.062550,1.485700,2.280150,3.515800,7.40665,10.325300,10.120300,9.82185,0.160500
std,1.277015,1.09966,1.090184,0.726266,0.833438,0.786741,0.90319,1.026564,1.123966,0.892544,1.280097,1.391275,9.94318,3.766443,4.188055,4.95591,0.367078
min,15.000000,0.00000,0.000000,1.000000,1.000000,0.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000,1.000000,-2.000000,-2.00000,0.000000
25%,16.000000,2.00000,2.000000,1.000000,1.000000,0.000000,4.00000,3.000000,2.000000,1.000000,1.000000,3.000000,2.00000,8.000000,7.000000,7.00000,0.000000
50%,17.000000,3.00000,3.000000,1.000000,2.000000,0.000000,4.00000,3.000000,3.000000,1.000000,2.000000,4.000000,4.00000,10.000000,10.000000,10.00000,0.000000
75%,18.000000,4.00000,3.000000,2.000000,2.000000,0.000000,5.00000,4.000000,4.000000,2.000000,3.000000,5.000000,9.00000,13.000000,13.000000,13.00000,0.000000
max,22.000000,4.00000,4.000000,4.000000,4.000000,3.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,75.00000,20.000000,20.000000,21.00000,1.000000


Drop some features

In [8]:
df.drop(columns=['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic','G3'], inplace=True)

Import scikit-learn and build a random forest classifer

encoding categorical features using LabelBinarizer

from sklearn.preprocessing import LabelBinarizer

Names = ['sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
for col in Names: 
    jobs_encoder = LabelBinarizer()
    jobs_encoder.fit(df[col])
    transformed = jobs_encoder.transform(df[col])
    ohe_df = pd.DataFrame(transformed)
    df = pd.concat([df, ohe_df], axis=1).drop([col], axis=1)


splitting and training the data 

In [9]:
from sklearn.ensemble import RandomForestClassifier as rf
import sklearn
from sklearn.model_selection import train_test_split

dependent_variable = 'qual_student'
X = df[df.columns.difference([dependent_variable])]
y = df[dependent_variable]

#splitting the train and test sets
X_train, X_test, y_train,y_test= train_test_split(X, y, test_size=0.5)
    
clf = rf(n_estimators = 1000)
clf.fit(X_train, y_train)


RandomForestClassifier(n_estimators=1000)

getting the accuracy metric 

In [10]:
pred = clf.predict(X_test)
sklearn.metrics.f1_score(y_test, pred, average='binary')

0.9446045469100225

Let's export this model so we can use it in a microservice (flask api)

In [11]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf, '/Users/meghanatera/Desktop/17-313/fall-2021-hw4-a-mama/dockerfile/apps/bettermodel.pkl')

['/Users/meghanatera/Desktop/17-313/fall-2021-hw4-a-mama/dockerfile/apps/bettermodel.pkl']

In [12]:
pred = clf.predict(X_test)
sklearn.metrics.f1_score(y_test, pred, average='binary')

0.9446045469100225

In [21]:
type(X_test)
print(X_test.loc[[0]])
type(pred)

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [index]"

In [13]:
minG1Reject = 100
maxG1Reject = -1
minG2Reject = 100
maxG2Reject = -1
minG1Accept = 100
maxG1Accept = -1
minG2Accept = 100
maxG2Accept = -1

for i in range(len(pred)):
    g1 = X_test[i].G1
    g2 = X_test[i].G2
    if pred[i] == 0:    
        if g1>maxG1Reject:
            maxG1Reject = g1
        if g1<minG1Reject:
            minG1Reject = g1
        
        if g2>maxG2Reject:
            maxG2Reject = g2
        if g2<minG2Reject:
            minG2Reject = g2
            
    else:    
        if g1>maxG1Accept:
            maxG1Accept = g1
        if g1<minG1Accept:
            minG1Accept = g1
        
        if g2>maxG2Accept:
            maxG2Accept = g2
        if g2<minG2Accept:
            minG2Accept = g2

print(minG1Reject)
print(maxG1Reject)
print(minG2Reject)
print(maxG2Reject)
print(minG1Accept)
print(maxG1Accept)
print(minG2Accept)
print(maxG2Accept)

KeyError: 0

In [ ]:
iris = sklearn.datasets.load_iris()

In [ ]:
predict_fn_rf = lambda x:clf.predict_proba(x).astype(float)
X = X_train.values
explainer = lime.lime_tabular.LimeTabularExplainer(X,feature_names = X_train.columns,
                                                   class_names=['Accept','Reject'], kernel_width=5)

In [ ]:
choosen_instance = X_test.loc[[23]].values[0]
exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=10)
exp.show_in_notebook(show_all=False)

In [ ]:
%matplotlib inline
fig = exp.as_pyplot_figure()